# SLR: Merge bibliometric files and remove duplicates

## 0 How to use

This notebook processes bibliometric datasets from **Web of Science**, **Scopus**, and **ProQuest**. Follow these steps to preprocess, merge, deduplicate, and analyze the data.

**NOTE:** The code only supports **CSV** and **Excel** formats for the bibliometric data.

### Steps to Use:

1. **Provide the Filepaths:**
   - Assign the file paths for your datasets to the following variables in the designated cell:
     - `FILE_PATH_WOS` → Path to the Web of Science dataset.
     - `FILE_PATH_SD` → Path to the Scopus dataset.
     - `FILE_PATH_PQ` → Path to the ProQuest dataset.

2. **Run All Cells:**
   - Execute all the code cells in the notebook sequentially to process the data.

3. **View the Results:**
   - The notebook will display:
     - The number of studies before and after duplicate removal.
     - The number of duplicates removed.
     - The count of studies from each source (Web of Science, Scopus, ProQuest).
     - The complete deduplicated dataset for review.

In [ ]:
FILE_PATH_WOS = ""
FILE_PATH_SD = ""
FILE_PATH_PQ = ""

## 1 Imports

Import of all necessary libraries:

In [ ]:
import pandas as pd
import warnings
from unidecode import unidecode

Globally suppress the SettingsWithCopyWarning from the Pandas library:

In [ ]:
# Suppress the SettingWithCopyWarning globally
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

## 2 Read bibliometric files

The function *read_file* takes a file path (*filename*) as input and reads it into a pandas DataFrame. It supports two file formats:

- **CSV files**: If the file has a .csv extension, it is read using pd.read_csv.
- **Excel files**: If the file doesn't have a .csv extension, it attempts to read it as an Excel file using pd.read_excel. If this fails (e.g., due to an unsupported format), it prints an error message: "Wrong file format".

In [ ]:
def read_file(filename: str) -> pd.DataFrame:
    """
    Reads a file into a Pandas DataFrame. Supports CSV and Excel formats.

    Args:
        filename (str): The path to the file to read.

    Returns:
        pd.DataFrame: A DataFrame containing the data from the file.

    Raises:
        ValueError: If the file format is unsupported or cannot be read.
    """

    # Check if the file is a CSV by its extension
    if (filename.endswith(".csv")):
        return pd.read_csv(filename)
    else:
        # Attempt to read as an Excel file if not a CSV
        try:
            return pd.read_excel(filename)
        except:
            # Print an error message if the file format is unsupported or cannot be read
            print("Wrong file format")

The *read_files* function is designed to read the three data files into Pandas DataFrames. It uses the *read_file* function to handle the file reading. The function returns a tuple containing these three DataFrames.

In [ ]:
def read_files() -> tuple:
    """
    Reads three specific bibliometric datasets into Pandas DataFrames.

    Uses the `read_file` function to read the bibliometric files specified by the following variables:
    - FILE_PATH_WOS: Path to the Web of Science dataset.
    - FILE_PATH_SD: Path to the Scopus dataset.
    - FILE_PATH_PQ: Path to the ProQuest dataset.

    Returns:
        tuple: A tuple containing three DataFrames:
            - df_wos (pd.DataFrame): DataFrame containing Web of Science data.
            - df_scopus (pd.DataFrame): DataFrame containing Scopus data.
            - df_proquest (pd.DataFrame): DataFrame containing ProQuest data.
    """
    
    df_wos = read_file(FILE_PATH_WOS)
    df_scopus = read_file(FILE_PATH_SD)
    df_proquest = read_file(FILE_PATH_PQ)

    return df_wos, df_scopus, df_proquest

## 3 Adjust DataFrame Structure

This code contains three functions that preprocess bibliometric datasets from the different sources — *Web of Science*, *Scopus*, and *ProQuest* — to standardize and clean the data for analysis. Each function takes a Pandas DataFrame as input and returns a cleaned DataFrame with consistent column names, filtered content, and transformed values.

More precisely, for each DataFrame, the code adjust the format and structure, so that all DataFrames are identical, allowing for merging later on.

In [ ]:
def adjust_wos(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses and standardizes bibliometric data from the Web of Science database.

    Args:
        df (pd.DataFrame): The raw Web of Science data as a Pandas DataFrame.

    Returns:
        pd.DataFrame: A cleaned and standardized DataFrame containing the following columns:
            - Authors: Author names (spaces removed, lowercase).
            - Title: Article title (lowercase).
            - Year: Year of publication.
            - Database: Fixed value 'Web of Science'.
            - Citations: Number of citations.
    """

    # Add column specifying database
    df["Database"] = "Web of Science"

    # Only keep relevant columns: authors, title and year
    df = df[["Author Full Names", "Article Title", "Publication Year", "Database", "Times Cited, WoS Core"]]

    # Rename columns to standard names
    df = df.rename(columns={"Author Full Names": "Authors", "Article Title": "Title",
                            "Publication Year": "Year", "Times Cited, WoS Core": "Citations"})

    # Remove spaces
    df["Authors"] = df["Authors"].str.replace(' ', '', regex=False)

    # Turn strings to lowercase
    df["Authors"] = df["Authors"].str.lower()
    df["Title"] = df["Title"].str.lower()

    return df

In [ ]:
def adjust_scopus(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses and standardizes bibliometric data from the Scopus database.

    Args:
        df (pd.DataFrame): The raw Scopus data as a Pandas DataFrame.

    Returns:
        pd.DataFrame: A cleaned and standardized DataFrame containing the following columns:
            - Authors: Author names (spaces and parentheses removed, lowercase).
            - Title: Article title (lowercase).
            - Year: Year of publication.
            - Database: Source of the data (formerly 'Source'), which is the fixed value 'Scopus'.
            - Citations: Number of citations (formerly 'Cited by').
    """

    # Only keep relevant columns: authors, title and year
    df = df[["Author full names", "Title", "Year", "Source", "Cited by"]]

    # Rename columns to standard names
    df = df.rename(columns={"Author full names": "Authors", "Cited by": "Citations", "Source": "Database"})

    # Remove spaces
    df["Authors"] = df["Authors"].str.replace(' ', '', regex=False)

    # Turn strings to lowercase
    df["Authors"] = df["Authors"].str.lower()
    df["Title"] = df["Title"].str.lower()

    # Remove parentheses and their contents in column "Authors"
    df["Authors"] = df["Authors"].str.replace(r'\(.*?\)', '', regex=True).str.strip()

    return df

In [ ]:
def adjust_proquest(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocesses and standardizes bibliometric data from the ProQuest database.

    Args:
        df (pd.DataFrame): The raw ProQuest data as a Pandas DataFrame.

    Returns:
        pd.DataFrame: A cleaned and standardized DataFrame containing the following columns:
            - Authors: Author names (spaces removed, lowercase).
            - Title: Article title (lowercase).
            - Year: Year of publication (extracted from 'PubDate').
            - Database: Source of the data (formerly 'Source'), which is the fixed value 'ProQuest'.
            - Citations: Number of citations (formerly 'Cited by').
    """

    # Only keep relevant columns: authors, title and year
    df = df[["Author", "Title", "PubDate", "Source", "Cited by"]]

    # Extract year from column "PubDate"
    df["PubDate"] = pd.to_datetime(df["PubDate"])
    df["PubDate"] = df["PubDate"].dt.year

    # Rename columns to standard names
    df = df.rename(columns={"Author": "Authors", "PubDate": "Year", "Cited by": "Citations", "Source": "Database"})

    # Remove spaces
    df["Authors"] = df["Authors"].str.replace(' ', '', regex=False)

    # Turn strings to lowercase
    df["Authors"] = df["Authors"].str.lower()
    df["Title"] = df["Title"].str.lower()

    return df

## 4 Merge DataFrame to a single DataFrame

The *merge_dfs* function combines data from the three Pandas DataFrames — *df_wos*, *df_scopus*, and *df_proquest* — into a single DataFrame using Pandas' concat function. It creates a unified dataset by stacking the rows from the input DataFrames.

Hence, the merged DataFrame contains all the studies.

In [ ]:
def merge_dfs(df_wos: pd.DataFrame, df_scopus: pd.DataFrame, df_proquest: pd.DataFrame) -> pd.DataFrame:
    """
    Merges three bibliometric DataFrames into a single DataFrame.

    Args:
        df_wos (pd.DataFrame): DataFrame containing Web of Science data.
        df_scopus (pd.DataFrame): DataFrame containing Scopus data.
        df_proquest (pd.DataFrame): DataFrame containing ProQuest data.

    Returns:
        pd.DataFrame: A merged DataFrame containing all rows from the three input DataFrames, 
                      with reset index for consistency.
    """
    
    return pd.concat([df_wos, df_scopus, df_proquest], ignore_index=True)

## 5 Remove Duplicates

The *rem_duplicates* function removes duplicate records from the merged Pandas DataFrame based on the combination of *Authors*, *Title*, and *Year* columns.

It ensures consistent text formatting by normalizing special characters and reorders the columns for better readability. The function also provides a summary of the number of unique rows compared to the original dataset.

In [ ]:
def rem_duplicates(df: pd.DataFrame) -> tuple:
    """
    Removes duplicate records from a DataFrame and provides a summary.

    Args:
        df (pd.DataFrame): The input DataFrame containing bibliometric data.

    Returns:
        tuple: A tuple containing:
            - df_uniques (pd.DataFrame): DataFrame with duplicates removed.
            - df_uniques.shape[0]: Number of unique rows in the DataFrame.
            - df.shape[0]: Number of rows in the original DataFrame.
    """
    
    # Handle variations in special characters
    df["Authors"] = df["Authors"].apply(unidecode)
    df["Title"] = df["Title"].apply(unidecode)

    # Remove duplicates
    df = df.sort_values(by="Citations", ascending=False)
    df_uniques = df.drop_duplicates(["Authors", "Title", "Year"], keep="first")

    # Reorder columns to make "Title" the first column
    df_uniques = df_uniques[["Title"] + [col for col in df_uniques.columns if col != "Title"]]

    df_uniques = df_uniques.sort_values(by="Title")

    return df_uniques, df_uniques.shape[0], df.shape[0]

## 6 Main function

The *main* function performs all necessary steps and calls all functions.

More precisely, the function orchestrates the end-to-end process of reading, preprocessing, merging, deduplicating, and summarizing bibliometric data from multiple sources: Web of Science, Scopus, and ProQuest. It ensures data consistency and provides a detailed summary of the dataset before and after removing duplicates.

In [ ]:
def main():
    """
    Main function to process and summarize bibliometric data from multiple sources.

    Workflow:
    1. Read raw data from Web of Science, Scopus, and ProQuest.
    2. Preprocess and standardize the individual datasets.
    3. Merge the datasets into a unified DataFrame.
    4. Remove duplicate studies and summarize results.
    5. Print the deduplicated DataFrame and summary statistics.

    Outputs:
        - Number of studies before and after deduplication.
        - Number of duplicates removed.
        - Breakdown of studies by source (Web of Science, Scopus, ProQuest).
        - Full deduplicated DataFrame.
    """
    
    df_wos, df_scopus, df_proquest = read_files()
    
    df_wos_adj = adjust_wos(df_wos)
    df_scopus_adj = adjust_scopus(df_scopus)
    df_proquest_adj = adjust_proquest(df_proquest)

    df_merged = merge_dfs(df_wos_adj, df_scopus_adj, df_proquest_adj)

    df_uniques, df_uniques_size, df_size = rem_duplicates(df_merged)

    df_wos = df_uniques[df_uniques["Database"] == "Web of Science"]
    df_scopus = df_uniques[df_uniques["Database"] == "Scopus"]
    df_proquest = df_uniques[df_uniques["Database"] == "ProQuest"]

    print(f"Number of studies before removal of duplicates studies: {df_size}")
    print(f"Number of studies removed: {df_size - df_uniques_size}")
    print(f"Number of studies after automatic removal of duplicates studies (automatic): {df_uniques_size}")
    print("------------------------------")
    print(f"Number of studied from Web of Science: {df_wos.shape[0]}")
    print(f"Number of studied from Scopus: {df_scopus.shape[0]}")
    print(f"Number of studied from ProQuest: {df_proquest.shape[0]}")
    print("------------------------------")
    print()
    print(df_uniques.to_string())


In [ ]:
main()